In [1]:
from keras.models import load_model


Using TensorFlow backend.


In [2]:
modelFile = '../model/keras/model/facenet_keras.h5'
# faceNetModel = load_model(modelFile)

In [3]:
faceNetModel.summary()

NameError: name 'faceNetModel' is not defined

In [4]:
facenet_model = load_model(modelFile)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/shibin/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-wa2tqe89
  Running setup.py bdist_wheel for keras-contrib ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-qpdxk817/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [55]:
from keras_contrib.applications import DenseNet
import keras.backend as K
import keras 
from keras.models import Model
from keras.layers import AveragePooling2D, Dense, Flatten,BatchNormalization, Input, Lambda, Concatenate
import keras.optimizers as optimizer

def loss_fun(label, pred, delta = 0.1):
    ## label is of the form [first_pred, second_pred, last_pred]
    #i label means the i is different from the rest of the pair
    x,y,z  = pred[0], pred[1], pred[2]
    
    ## y and z is similar
    if label == 1:
        return K.maximum(0.0, K.square(y-z) - K.square(y-x) + delta) + K.maximum(0.0, K.square(y-z) - K.square(z- x) + delta) 
    elif label == 2:
        return K.maximum(0.0, K.square(x-z) - K.square( x- y) + delta) + K.maximum(0.0, K.square(x - z) - K.square(y- z) + delta)
    
    else:
        return K.maximum(0.0, K.square(x- y) - K.square( x -z) + delta) + K.maximum(0.0, K.square(x- y) - K.square( y- z ) + delta)
    
        
def accuracy_c(y_true, y_pred):
    s = loss_fun(K.cast(y_true, 'int32'), y_pred)
    return K.equal(s, 0.0)
        
        
    
    
def create_new_model(input_shape):
#     facenet_model = load_model(modelFile)
    inputs = Input(shape=input_shape)
    #i = facenet_model(inputs)
    #print(facenet_model.get_input_at(0).shape)
    model1 = Model(input=facenet_model.get_input_at(0) , output = facenet_model.get_layer('add_15').output)
    
    inter_shape =  model1.output_shape[1:]
    print(inter_shape)
    #model2 = Model(input=model1.input1, outputs = inter_layer);
    #inter_shape = 
    print(inter_shape)
    dense = DenseNet(
                     input_shape = inter_shape, nb_filter = 600, include_top = False,  depth = 1, nb_dense_block = 2, growth_rate = 32)(model1.output)
    #d.layers.pop()
    #d.layers.pop()
    #d.layers.pop()
    #d = d.layers.pop()
    #dense.summary()
    #model = d
    #model.summary()
    x = AveragePooling2D(pool_size=(4,4), name='cutome_avg_pool_1')(dense)
    x = Flatten(name  = 'cutome_flatten_2')(x)
    x = Dense(512, activation='relu', name = 'custome_dense_dense_3')(x)
    x = BatchNormalization(name='cutome_batch_normalization_4')(x)
    x = Dense(16, activation='relu', name = 'custome_dense_5')(x)
    #x = Dense(1, activation='relu', name = 'custome_dense_output_1')(x)
    x = Lambda(lambda  x: K.l2_normalize(x,axis=1), name = 'custome_l2_normalize_6')(x)
    #fin = d(inter_layer.output)
    
    model = Model(inputs=[model1.input], outputs=x)
    input_1 = Input( shape = input_shape)
    input_2 = Input( shape = input_shape)
    input_3 = Input( shape = input_shape)
    
    l1 = model(input_1)
    l2 = model(input_2)
    l3 = model(input_3)
    
    x = Concatenate()([l1, l2, l3])
    final_model = Model(inputs=[input_1, input_2, input_3], outputs=x)
    
    
    final_model.compile(optimizer=optimizer.Adam( lr = 0.0005),
              loss= loss_fun, metrics = [accuracy_c] )
    return final_model
    

In [56]:
embedding_model = create_new_model((160,160,3))

/home/shibin/softwares/anaconda2/envs/aiproject/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ad...)`
/home/shibin/.local/lib/python3.6/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 896 input channels.
  str(input_shape[-1]) + ' input channels.')


(8, 8, 896)
(8, 8, 896)


In [57]:
embedding_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_113 (InputLayer)          (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_114 (InputLayer)          (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_115 (InputLayer)          (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
model_61 (Model)                (None, 16)           15142240    input_113[0][0]                  
                                                                 input_114[0][0]                  
          

In [ ]:
# example of loading an image with the Keras API
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
test_img = img_to_array(load_img('test.jpg'))
#test_imgs = (test_img, test_img, test_img)
inp = np.expand_dims(test_img, axis = 0)
embedding_model.predict([inp, inp, inp])

In [71]:
embedding_model.input

[<tf.Tensor 'input_113:0' shape=(?, 160, 160, 3) dtype=float32>,
 <tf.Tensor 'input_114:0' shape=(?, 160, 160, 3) dtype=float32>,
 <tf.Tensor 'input_115:0' shape=(?, 160, 160, 3) dtype=float32>]

In [73]:
inp.shape

(1, 3, 160, 160, 3)